In [37]:
import os
import subprocess

def get_video_duration(file_path):
    """
    Get the duration of a video file using ffmpeg.
    
    :param file_path: Path to the video file.
    :return: Duration of the video in seconds.
    """
    result = subprocess.run(
        ["ffprobe", "-v", "error", "-show_entries", "format=duration", 
         "-of", "default=noprint_wrappers=1:nokey=1", file_path],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )
    
    try:
        return float(result.stdout.strip())
    except ValueError:
        return 0.0

def format_duration(seconds):
    """
    Format the duration from seconds to hours and minutes.
    
    :param seconds: Duration in seconds.
    :return: Formatted duration as a string.
    """
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    return f"{hours} hours {minutes} minutes"

def calculate_total_duration(directory):
    """
    Calculate the total duration of all video files in a directory and its subdirectories.
    Print only for the first-level subdirectories.
    
    :param directory: The root directory to search for video files.
    :return: The total duration in seconds.
    """
    total_duration = 0.0
    video_extensions = ('.mp4', '.mkv', '.avi', '.mov', '.flv', '.wmv', '.webm')  # Add more as needed
    
    print(f"📂 Scanning Directory: {directory}")
    
    # First pass to calculate the total duration for the root directory itself
    root_duration = 0.0
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if os.path.isfile(file_path) and file.lower().endswith(video_extensions):
            root_duration += get_video_duration(file_path)
    
    total_duration += root_duration
    
    # Now walk through the directory and its subdirectories
    for dirpath, dirnames, _ in os.walk(directory):
        # Calculate for first-level subdirectories
        if dirpath == directory:
            for subdir in dirnames:
                subdir_path = os.path.join(directory, subdir)
                subdir_duration = 0.0
                for subdir_dirpath, _, subdir_filenames in os.walk(subdir_path):
                    for file in subdir_filenames:
                        file_path = os.path.join(subdir_dirpath, file)
                        if file.lower().endswith(video_extensions):
                            subdir_duration += get_video_duration(file_path)
                
                # Print duration for the first-level subdirectory, only the name
                if subdir_duration > 0:
                    print(f"📼 Total duration of video files in '{subdir}': {format_duration(subdir_duration)}")
                
                total_duration += subdir_duration

        # Break after processing the first level of subdirectories (but continue calculating for deeper levels)
        break
    
    return total_duration

if __name__ == "__main__":
    print("📊 Video Duration Calculator - By Abhijit 🧑‍💻\n" + "="*30)
    directory = input("Enter the directory path to scan for video files (e.g., /Users/username/Movies): ")
    
    # Ensure the path is correctly formatted for macOS
    if os.path.exists(directory):
        total_duration = calculate_total_duration(directory)
        print(f"🎉 Total duration of all video files: {format_duration(total_duration)}")
    else:
        print(f"❌ Error: Directory '{directory}' does not exist.")


📊 Video Duration Calculator - By Abhijit 🧑‍💻
📂 Scanning Directory: /Users/abhijit/Downloads/Courses
📼 Total duration of video files in 'Deep Learning for Computer Vision': 39 hours 33 minutes
📼 Total duration of video files in 'LLM - IIT Madras - Mitesh Khapra': 13 hours 44 minutes
📼 Total duration of video files in 'Andrej Karpathy- Neural Network': 22 hours 49 minutes
📼 Total duration of video files in 'LLM Bootcamp': 8 hours 13 minutes
📼 Total duration of video files in 'Practical Deep Learning for Coders': 16 hours 9 minutes
🎉 Total duration of all video files: 100 hours 30 minutes
